In [2]:
import torch
from torch.utils import data as data_
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch import optim
import ast


df = pd.read_csv(r"C:\Users\YuCheng_Ch\Desktop\rand_ans_que_merge_vectors.csv")
sentence = df['sentence'].values.tolist()
merged_victor = df['vector'].values.tolist()
merged_label = df['label'].values.tolist()

In [3]:



total_ele = []
max_num = 0
# for num, victor in enumerate(merged_victor):
#     temp_list = eval(victor.replace('[[','[').replace(']]',']'))
#     for ele in temp_list:
#         total_ele.append(ele)


max_num = max(merged_victor)


data_x = []
data_y = [] 
for num, list_ in enumerate(merged_victor):

    try:
        data_x.append(ast.literal_eval(list_))
    except:
        print(list_)

for num, label in enumerate(merged_label):
    try:
        data_y.append(int(label))
    except:
        print(label)



In [4]:
train_x, test_x, train_y, test_y = train_test_split(data_x,data_y, test_size=0.1)


train_x= torch.tensor(train_x, dtype=torch.float32)
# train_x = (train_x/max_num).cuda()
train_y= torch.tensor(train_y, dtype=torch.int64).cuda()

test_x = torch.tensor(test_x, dtype=torch.float32)
# test = (test_x/max_num).cuda()
test_y = torch.tensor(test_y, dtype=torch.int64).cuda()

#xy打包
train = torch.utils.data.TensorDataset(train_x, train_y)
test = torch.utils.data.TensorDataset(test_x, test_y)
#打包後迭帶
train_dataloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)


In [5]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        


        self.linear1 = nn.Linear(768,512)
        self.linear2 = nn.Linear(512,256)
        self.linear3 = nn.Linear(256, 128)        
        self.linear4 = nn.Linear(128, 64)
        self.linear5 = nn.Linear(64, 16)
        self.linear6 = nn.Linear(16, 8)
        self.output = nn.Linear(8, 2)          

    
    def forward (self, x ):
        x = self.linear1(x)
        x = F.relu (x)
        x = self.linear2(x)
        x = F.relu (x)
        x = self.linear3(x)
        x = F.relu (x)
        x = self.linear4(x)
        x = F.relu (x)
        x = self.linear5(x)   
        x = F.relu (x)
        x = self.linear6(x)   
        x = F.relu (x)
        x = self.output(x)
        x = F.softmax(x, dim = 1)
       
        return x
LR = 5e-5
##### 加權損失函數
class_counts = np.bincount(train_y.cpu().numpy())
class_weights = 1. / class_counts
weights = torch.tensor(class_weights, dtype=torch.float32).cuda()
loss_fun = nn.CrossEntropyLoss(weight=weights)
##### 加權損失函數


model = NN().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)








In [6]:
import os 
epochs = 100
save_interval = 25
log_interval = 25
save_directory = r'C:\Users\YuCheng_Ch\Desktop\datasets_accton\onetime_train'

# 確保儲存資料夾存在
os.makedirs(save_directory, exist_ok=True)

for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for tensor, target in train_dataloader:   
        tensor = tensor.cuda()
        target = target.cuda()
        optimizer.zero_grad()
        result = model(tensor)
        loss = loss_fun(result, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(result.data, 1)
        total_train += target.size(0)
        correct_train += (predicted == target).sum().item()

    if (epoch + 1) % log_interval == 0:
        average_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct_train / total_train
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {average_loss:.4f}, Accuracy: {train_accuracy:.2f}%')

    if (epoch + 1) % save_interval == 0:
        model.eval()
        correct = 0
        total = 0
        test_loss = 0.0

        with torch.no_grad():
            for tensor, target in test_dataloader:
                tensor = tensor.cuda()
                target = target.cuda()
                output = model(tensor)
                # for num in range(0,25):
                #     print(output[num][1]) #eval probability.
                loss = loss_fun(output, target)
                test_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        test_loss /= len(test_dataloader)
        print(f'Epoch [{epoch + 1}/{epochs}] - Accuracy on test data: {accuracy:.2f}%, Loss: {test_loss:.4f}')
        
        # Save the model
        model_path = os.path.join(save_directory, f'model_epoch_{epoch + 1}_acc_{accuracy:.2f}_loss_{test_loss:.4f}.pth')
        torch.save(model.state_dict(), model_path)
        print(f'Model saved to {model_path}')

 25%|██▌       | 25/100 [00:11<00:31,  2.40it/s]

Epoch [25/100], Loss: 0.4727, Accuracy: 83.23%
Epoch [25/100] - Accuracy on test data: 83.49%, Loss: 0.4775
Model saved to C:\Users\YuCheng_Ch\Desktop\datasets_accton\onetime_train\model_epoch_25_acc_83.49_loss_0.4775.pth


 50%|█████     | 50/100 [00:24<00:21,  2.33it/s]

Epoch [50/100], Loss: 0.4488, Accuracy: 85.48%
Epoch [50/100] - Accuracy on test data: 84.49%, Loss: 0.4557
Model saved to C:\Users\YuCheng_Ch\Desktop\datasets_accton\onetime_train\model_epoch_50_acc_84.49_loss_0.4557.pth


 75%|███████▌  | 75/100 [00:35<00:10,  2.32it/s]

Epoch [75/100], Loss: 0.4297, Accuracy: 87.67%
Epoch [75/100] - Accuracy on test data: 86.37%, Loss: 0.4447
Model saved to C:\Users\YuCheng_Ch\Desktop\datasets_accton\onetime_train\model_epoch_75_acc_86.37_loss_0.4447.pth


100%|██████████| 100/100 [00:47<00:00,  2.11it/s]

Epoch [100/100], Loss: 0.4253, Accuracy: 88.06%
Epoch [100/100] - Accuracy on test data: 87.64%, Loss: 0.4468
Model saved to C:\Users\YuCheng_Ch\Desktop\datasets_accton\onetime_train\model_epoch_100_acc_87.64_loss_0.4468.pth
